In [1]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [2]:
import pydss

## CIGRE LV European System

In [3]:
sys1 = pydss.pydss('cigre_lv_connected_3w.json')  # Load data
sys1.pf()                            # execute power flow 

In [4]:
%timeit pydss.pf_eval(sys1.params_pf)

The slowest run took 4.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 291 µs per loop


In [5]:
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

### Graph with obtained results

In [6]:
sys1.bokeh_tools()

p = figure(width=600, height=800,
           title='European LV Network (CIGRE)', 
           x_range=[-200,200], y_range=[-350, 50])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size=15, color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

### Interaction with powers

In [12]:
s_0_3pn = sys1.pq_3pn # 
s_0_3p  = sys1.pq_3p  # 

p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           x_range = [50,-300], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update(load_factor=1.0):

    sys1.pq_3pn = load_factor*s_0_3pn
    sys1.pq_3p = load_factor*s_0_3p
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [13]:
from ipywidgets import interact
interact(update, load_factor=(-1,1.2, 0.01))

### Interaction with compensator currents

In [15]:
p = figure(width=600, height=400,
           title='Voltage with compensation', 
           x_range = [50,-300], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)

i_a_0 = np.abs(sys1.I_node[28,0])
i_b_0 = np.abs(sys1.I_node[29,0])
i_c_0 = np.abs(sys1.I_node[30,0])
s_0 = sys1.pq_3pn # 

sys1.pf()

cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))

def update(i_a=0.0,i_b=0.0,i_c=0.0):

    sys1.I_node[28,0] = (i_a + i_a_0)*np.exp(1j*np.angle(sys1.V_node[28,0] -np.pi/2 ))
    sys1.I_node[29,0] = (i_b + i_b_0)*np.exp(1j*np.angle(sys1.V_node[29,0] -np.pi/2 ))
    sys1.I_node[30,0] = (i_c + i_c_0)*np.exp(1j*np.angle(sys1.V_node[30,0] -np.pi/2 ))    
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

show(p, notebook_handle=True)

In [16]:
interact(update, 
         i_a=(-100,100, 1),
         i_b=(-100,100, 1),
         i_c=(-100,100, 1))